In [ ]:
!pip install -q numpy pandas datasets transformers tqdm safetensors scikit-learn

In [ ]:
!pip3 install -q -U datasets flash-attn transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 73.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00

In [ ]:
!pip3 install -q torch torchvision torchaudio

In [ ]:
!pip install -q -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 76.8 MB/s eta 0:00:00


# Load Reward Model

In [ ]:
import torch

# Set up CUDA optimizations for faster computation
torch.backends.cuda.matmul.allow_tf32 = (
    True  # Enable TensorFloat-32 matrix multiplication on CUDA
)
torch.backends.cudnn.allow_tf32 = (
    True  # Allow TensorFloat-32 in cuDNN for faster convolution operations
)

In [ ]:
REWARD_MODEL_REPO_NAME = "il-pugin/hse-prog-task-transformer-reward-model"

In [ ]:
DEVICE = 0
RANDOM_STATE = 42

In [ ]:
attributes = [
    'helpful',
    'does_not_reveal_answer',
    'does_not_contain_errors',
    'only_relevant_info',
]

In [ ]:
from transformers.modeling_utils import PretrainedConfig, PreTrainedModel
from transformers import AutoConfig, AutoModel
from torch import nn

# Configuration
class RewardModelConfig(PretrainedConfig):
    model_type = "reward_model"

    def __init__(self, base_model_config=None, attributes=None, **kwargs):
        # Store the original transformer config
        self.base_model_config = base_model_config or {}
        self.attributes = attributes or []
        # Initialize with combined parameters
        super().__init__(**{**self.base_model_config, **kwargs})

# Model class
class RewardModel(PreTrainedModel):
    config_class = RewardModelConfig

    def __init__(self, config):
        super().__init__(config)

        # Recreate base model config from stored parameters
        base_config = AutoConfig.for_model(**config.base_model_config)

        # Initialize base transformer
        self.transformer = AutoModel.from_config(base_config)

        # Initialize regression heads
        self.regression_heads = nn.Linear(
            base_config.hidden_size,
            len(config.attributes),
            bias=False
        )

    def forward(self, input_ids, attention_mask=None):
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        last_hidden = outputs.last_hidden_state[:, -1, :]
        return self.regression_heads(last_hidden)

# 3. Registration (only needed when saving/loading)
AutoConfig.register(RewardModelConfig.model_type, RewardModelConfig)
AutoModel.register(RewardModelConfig, RewardModel)

In [ ]:
import torch

reward_model = RewardModel.from_pretrained(
    REWARD_MODEL_REPO_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

config.json:   0%|          | 0.00/3.23k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.7k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
device = f"cuda:{DEVICE}"

In [ ]:
from transformers import AutoTokenizer

reward_model_tokenizer = AutoTokenizer.from_pretrained(REWARD_MODEL_REPO_NAME)

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

# Load Dataset

In [ ]:
import os
import torch
import datasets
import numpy as np
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from safetensors.torch import save_file
import pandas as pd

In [ ]:
# Define attributes (reward objectives)
attributes = [
    'helpful',
    'does_not_reveal_answer',
    'does_not_contain_errors',
    'only_relevant_info',
]


DATASET_PATH = "il-pugin/hse-porg-task-transformer-lora-dataset"
N_SHARDS = 1
SHARD_IDX = 1


In [ ]:
# Load the specified dataset and prepare it for processing
ds_train_val = datasets.load_dataset(DATASET_PATH, split="train")
ds_test = datasets.load_dataset(DATASET_PATH, split="test")

data-00000-of-00001.arrow:   0%|          | 0.00/306k [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/75.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12 [00:00<?, ? examples/s]

In [ ]:
def transform_to_ppo_format(example):
    # Extract the user prompt (the content with role="user")
    user_prompt = next(message["content"] for message in example["messages"]
                      if message["role"] == "user")

    return {
        "query": user_prompt,
        "prompt_id": example["prompt_id"],
        "dataset": example["dataset"],
        "system_message": next(message["content"] for message in example["messages"]
                             if message["role"] == "system"),
    }

In [ ]:
ds_train_val = ds_train_val.map(transform_to_ppo_format, remove_columns='messages')
ds_test = ds_test.map(transform_to_ppo_format, remove_columns='messages')

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [ ]:
if N_SHARDS > 1:
    ds_train_val = ds_train_val.shard(
        num_shards=N_SHARDS, index=SHARD_IDX - 1
    )  # Divide dataset into shards if needed
    ds_test = ds_test.shard(
        num_shards=N_SHARDS, index=SHARD_IDX - 1
    )

# Define PEFT Pipeline

In [ ]:
import numpy as mp
import torch


def tokenize_for_rm(ds: datasets.Dataset, rm_tokenizer):
  # Iterate over each example in the dataset
  for example in ds:
    # Format the conversation messages using the tokenizer's chat template without tokenization
    conv_formatted = rm_tokenizer.apply_chat_template(
        example["messages"], tokenize=False, add_generation_prompt=False
    ).replace(rm_tokenizer.bos_token, "")
    # Tokenize the formatted conversation and move tensors to the specified device
    conv_tokenized = rm_tokenizer(conv_formatted, return_tensors="pt")
    yield conv_tokenized

In [ ]:
!pip install -q peft

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/60.3k [00:00<?, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

...

model-00016-of-000017.safetensors:   0%|          | 0.00/8.69G [00:00<?, ?B/s]

model-00017-of-000017.safetensors:   0%|          | 0.00/10.5G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

trainable params: 32,768,000 || all params: 70,586,474,496 || trainable%: 0.0464


In [ ]:
!pip install --no-build-isolation trl==0.7.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 12.7 MB/s eta 0:00:00


In [ ]:
from trl import PPOConfig

config = PPOConfig(
    exp_name="DeepSeek-R1-Distill-Llama-70B-LoRA",
    # task_name
    # model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
    # reward_model="sfairXC/FsfairX-LLaMA3-RM-v0.1",
    output_dir="./deepseek-r1-lora-rlhf",
    learning_rate=1e-5,
    max_grad_norm=0.3,
    num_train_epochs=3,
    batch_size=25,
    logging_dir="./logs",
    logging_first_step=True,
    report_to="none",
    remove_unused_columns=False,
    # early_stopping=True,
    seed=RANDOM_STATE,
    # is_peft_model=True,
)

In [ ]:
from trl import PPOTrainer

# Initialize PPO trainer
ppo_trainer = PPOTrainer(
    model=model,
    ref_model=None,
    tokenizer=tokenizer,
    train_dataset=ds_train_val,
    args=config,
)

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 128,
}

# Training Loop

In [ ]:
for epoch in range(10):
    for batch_idx, batch in enumerate(ppo_trainer.dataloader):
        prompts = batch["prompt"]

        # Tokenize entire batch
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
        ).to(model.device)

        # Generate responses for entire batch
        outputs = model.generate(
            **inputs,
            **generation_kwargs
        )

        # Extract responses (remove prompt)
        response_tensors = outputs[:, inputs.input_ids.size(1):]

        # Decode responses for reward model
        responses = [
            tokenizer.decode(
                output,
                skip_special_tokens=True
            ) for output in outputs
        ]

        # Get rewards for entire batch
        reward_inputs = reward_model_tokenizer(
            responses,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(reward_model.device)

        with torch.no_grad():
            reward_outputs = reward_model(**reward_inputs)

        # 1. Modified Reward Aggregation with Custom Weights
        reward_weights = torch.tensor([0.3, 0.2, 0.3, 0.2],
                                    device=reward_outputs.logits.device)
        batch_rewards = torch.einsum('ij,j->i',
                                   reward_outputs.logits,
                                   reward_weights)

        # 2. Perplexity Calculation
        with torch.no_grad():
            # Get model's logits for generated responses
            model.eval()
            response_logits = model(
                input_ids=outputs,
                attention_mask=(outputs != tokenizer.pad_token_id).long()
            ).logits

            # Calculate cross-entropy loss
            shift_logits = response_logits[..., :-1, :].contiguous()
            shift_labels = outputs[..., 1:].contiguous()

            loss_fct = torch.nn.CrossEntropyLoss(
                reduction='none',
                ignore_index=tokenizer.pad_token_id
            )

            # Reshape and calculate perplexity
            B, T = shift_labels.size()
            loss = loss_fct(
                shift_logits.view(B*T, -1),
                shift_labels.view(-1)
            ).view(B, T)

            # Mask out padding
            mask = (shift_labels != tokenizer.pad_token_id).float()
            avg_loss = (loss * mask).sum(dim=1) / mask.sum(dim=1)
            perplexity = torch.exp(avg_loss)
            model.train()

        # Prepare tensors for PPO
        query_tensors = [inputs.input_ids[i] for i in range(len(prompts))]
        response_tensors = [response_tensors[i] for i in range(len(prompts))]
        rewards = [batch_rewards[i].unsqueeze(0) for i in range(len(prompts))]

        # PPO Update Step
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

        # Monitoring with perplexity
        print(f"""
        Epoch: {epoch+1}
        Batch: {batch_idx+1}/{len(ppo_trainer.dataloader)}
        Mean Reward: {batch_rewards.mean().item():.4f}
        Mean Perplexity: {perplexity.mean().item():.2f}
        Max Perplexity: {perplexity.max().item():.2f}
        Min Perplexity: {perplexity.min().item():.2f}
        """)

# Save final model
model.save_pretrained("./trained-deepseek-r1-lora")